EXTRACCIÓN DEL SSI, PIM, COSTO, DEV ACUMULADO Y DEV ACTUAL

In [4]:
!pip install selenium webdriver-manager openpyxl pandas

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install playwright
!playwright install chromium

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/35.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/35.6 MB ? eta -:--:--
   ---------

ERROR: Exception:
Traceback (most recent call last):
  File "D:\Anaconda\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "D:\Anaconda\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\Lib\socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\Lib\ssl.py", line 1251, in recv_int

## SCRIPT PARA EXTRAER COSTO, PIM, PIA,DEVENGADO, ETC.. DEL ENLACE https://ofi5.mef.gob.pe/ssi/ VENTANA EMERGENTE O VER RESUMEN

In [4]:
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ================================================================
# CONFIGURACIÓN OPTIMIZADA
# ================================================================
RUTA_ENTRADA = r"C:\Users\user\Downloads\mef1.xlsx"
RUTA_SALIDA = r"C:\Users\user\Downloads\mef montos.xlsx"
MAX_REINTENTOS = 2
MODO_VISIBLE = True
GUARDAR_CADA = 5

# Timeouts optimizados
TIMEOUT_PAGINA = 20
TIMEOUT_ELEMENTO = 10
TIMEOUT_MODAL = 10

# ================================================================
# FUNCIONES DE CHECKPOINT MEJORADAS 🆕
# ================================================================
def cargar_progreso():
    """Carga progreso previo si existe"""
    if os.path.exists(RUTA_SALIDA):
        try:
            df = pd.read_excel(RUTA_SALIDA)
            print(f"📥 Progreso encontrado: {len(df)} CUIs ya procesados")
            return df.to_dict('records')
        except:
            pass
    return []

def obtener_pendientes(completa, procesados):
    """Calcula CUIs pendientes O con NO DISPONIBLE"""
    if not procesados:
        print(f"📋 Todos los CUIs pendientes: {len(completa)}")
        return completa, []
    
    # Crear diccionario de procesados
    procesados_dict = {str(r['CUI']): r for r in procesados}
    
    # 1. CUIs nunca procesados
    cuis_nuevos = [cui for cui in completa if str(cui) not in procesados_dict]
    
    # 2. CUIs con "NO DISPONIBLE" en Costo Inversión Total
    cuis_reintentar = []
    for cui in completa:
        if str(cui) in procesados_dict:
            registro = procesados_dict[str(cui)]
            costo = str(registro.get('Costo Inversión Total (a)', '')).strip().upper()
            
            # Si tiene "NO DISPONIBLE" en el costo, reintentarlo
            if costo == 'NO DISPONIBLE' or costo == '':
                cuis_reintentar.append(cui)
    
    # Combinar ambos
    pendientes = cuis_nuevos + cuis_reintentar
    
    if pendientes:
        print(f"📋 Nuevos: {len(cuis_nuevos)} | Reintentos: {len(cuis_reintentar)} | Total pendiente: {len(pendientes)}")
    
    return pendientes, procesados_dict

def guardar(resultados):
    """Guarda progreso"""
    try:
        pd.DataFrame(resultados).to_excel(RUTA_SALIDA, index=False)
    except Exception as e:
        print(f"⚠️ Error guardando: {e}")

# ================================================================
# LEER CUIs
# ================================================================
print("📂 Cargando CUIs...")
df_cui = pd.read_excel(RUTA_ENTRADA)
lista_completa = df_cui['CUI'].astype(str).tolist()

resultados_previos = cargar_progreso()
lista_cuis, procesados_dict = obtener_pendientes(lista_completa, resultados_previos)

if not lista_cuis:
    print("🎉 ¡Todos los CUIs ya fueron procesados exitosamente!")
    print("💡 Si quieres reintentar CUIs fallidos, verifica que tengan 'NO DISPONIBLE' en el archivo.")
    exit()

# Inicializar resultados con los previos (para mantener los exitosos)
resultados = resultados_previos.copy()

# ================================================================
# CONFIGURAR NAVEGADOR OPTIMIZADO
# ================================================================
def crear_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()

    if not MODO_VISIBLE:
        options.add_argument("--headless=new")
    else:
        options.add_argument("--start-maximized")

    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-logging")
    options.add_argument("--log-level=3")
    options.add_argument("--silent")

    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.media_stream": 2,
        "profile.default_content_setting_values.geolocation": 2,
    }
    options.add_experimental_option("prefs", prefs)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.page_load_strategy = 'normal'

    driver = Chrome(service=service, options=options)
    driver.set_page_load_timeout(TIMEOUT_PAGINA)
    driver.set_script_timeout(15)
    return driver

driver = crear_driver()
print(f"⚡ Procesando {len(lista_cuis)} CUIs pendientes\n")

# ================================================================
# CERRAR VENTANAS EMERGENTES
# ================================================================
def cerrar_ventanas_emergentes(driver):
    try:
        driver.execute_script("""
            document.querySelectorAll('.modal.show').forEach(m => {
                m.style.display='none';
                m.classList.remove('show');
            });
            document.querySelectorAll('.modal-backdrop').forEach(b => b.remove());
            document.body.classList.remove('modal-open');
            document.body.style.overflow='';
            document.body.style.paddingRight='';
        """)
    except:
        pass

# ================================================================
# ESPERAR Y VERIFICAR MODAL
# ================================================================
def esperar_modal_visible(driver, timeout=10):
    try:
        Wait(driver, timeout).until(
            EC.visibility_of_element_located((By.ID, "divResumenCont"))
        )
        time.sleep(0.3)
        
        tiene_contenido = driver.execute_script("""
            var modal = document.getElementById('divResumenCont');
            if (!modal) return false;
            if (modal.children.length === 0) return false;
            if (modal.offsetParent === null) return false;
            
            var tieneValores = document.getElementById('td_mtototal2_r') !== null &&
                               document.getElementById('val_pim_r') !== null;
            
            return tieneValores;
        """)
        
        if not tiene_contenido:
            time.sleep(0.5)
            tiene_contenido = driver.execute_script("""
                return document.getElementById('td_mtototal2_r') !== null;
            """)
        
        return tiene_contenido
        
    except TimeoutException:
        return False
    except Exception as e:
        print(f"⚠️ Error: {str(e)[:20]}", end=" ")
        return False

# ================================================================
# EXTRACCIÓN DE DATOS COMPLETO PARA 2026
# ================================================================
def extraer_datos_modal(driver):
    try:
        datos = driver.execute_script("""
            function getText(id) {
                try {
                    var elem = document.getElementById(id);
                    if (!elem) return 'NO DISPONIBLE';
                    var texto = elem.textContent || elem.innerText || '';
                    return texto.trim() || 'NO DISPONIBLE';
                } catch(e) {
                    return 'NO DISPONIBLE';
                }
            }

            function getPIA2026() {
                try {
                    var tabla = document.getElementById('tb_hist_anual_res');
                    if (!tabla) return 'NO DISPONIBLE';
                    
                    var filas = tabla.querySelectorAll('tr');
                    for (var i = 0; i < filas.length; i++) {
                        var celdas = filas[i].querySelectorAll('td');
                        if (celdas.length >= 2) {
                            var anio = celdas[0].textContent.trim();
                            if (anio === '2026') {
                                var valor = celdas[1].textContent.trim();
                                return valor || 'NO DISPONIBLE';
                            }
                        }
                    }
                } catch(e) {
                    console.error('Error getPIA2026:', e);
                }
                return 'NO DISPONIBLE';
            }

            var resultado = {
                costo_total: getText('td_mtototal2_r'),
                pia_2026: getPIA2026(),
                pim_2026: getText('val_pim_r'),
                devengado_acum: getText('val_efin_r'),
                devengado_2026: getText('val_avan_r'),
                fecha_inicio: getText('fec_iniejec_r'),
                fecha_fin: getText('fec_finejec_r'),
                fecha_primer_devengado: getText('pridev_r')
            };
            
            return resultado;
        """)

        valores_validos = sum(1 for v in datos.values() if v != 'NO DISPONIBLE')
        
        if valores_validos < 3:
            print(f"⚠️ Pocos datos ({valores_validos}/8)", end=" ")
            return None
            
        return datos

    except Exception as e:
        print(f"⚠️ Error: {str(e)[:20]}", end=" ")
        return None

# ================================================================
# PROCESAR CUI
# ================================================================
def procesar_cui(cui, intento=1):
    try:
        driver.get("https://ofi5.mef.gob.pe/ssi/")
        
        input_box = Wait(driver, TIMEOUT_ELEMENTO).until(
            EC.element_to_be_clickable((By.ID, "txt_cu"))
        )
        input_box.clear()
        input_box.send_keys(cui)
        
        btn_buscar = Wait(driver, TIMEOUT_ELEMENTO).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "btn_bus"))
        )
        btn_buscar.click()

        Wait(driver, TIMEOUT_ELEMENTO).until(
            EC.presence_of_element_located((By.ID, "td_cu"))
        )
        time.sleep(0.3)

        cerrar_ventanas_emergentes(driver)
        time.sleep(0.2)

        try:
            btn_resumen = Wait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "img[src*='resumen.png']"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", btn_resumen)
            time.sleep(0.2)
            driver.execute_script("arguments[0].click();", btn_resumen)
        except:
            driver.execute_script("""
                var imgs = document.querySelectorAll('img[src*="resumen.png"]');
                if (imgs.length > 0) {
                    imgs[0].click();
                }
            """)
        
        time.sleep(0.5)

        if not esperar_modal_visible(driver, TIMEOUT_MODAL):
            raise Exception("Modal no cargó")

        datos = extraer_datos_modal(driver)
        
        if not datos:
            raise Exception("Sin datos")

        try:
            driver.execute_script("""
                var btn = document.querySelector('button[data-bs-dismiss="modal"]');
                if (btn) btn.click();
            """)
            time.sleep(0.2)
        except:
            pass

        return datos

    except Exception as e:
        if intento < MAX_REINTENTOS:
            print(f"🔄{intento+1}...", end=" ")
            time.sleep(1)
            cerrar_ventanas_emergentes(driver)
            return procesar_cui(cui, intento + 1)
        else:
            raise e

# ================================================================
# ACTUALIZAR O AGREGAR RESULTADO
# ================================================================
def actualizar_resultado(cui, datos):
    """Actualiza un CUI existente o agrega uno nuevo"""
    # Buscar si ya existe en resultados
    for i, r in enumerate(resultados):
        if str(r['CUI']) == str(cui):
            # ACTUALIZAR el existente
            resultados[i] = {
                "CUI": cui,
                "Costo Inversión Total (a)": datos['costo_total'],
                "PIA 2026": datos['pia_2026'],
                "PIM 2026 (c)": datos['pim_2026'],
                "Devengado Acumulado al 2026 (b)": datos['devengado_acum'],
                "Devengado 2026 (d)": datos['devengado_2026'],
                "Fecha Inicio Ejecución": datos['fecha_inicio'],
                "Fecha Fin Ejecución": datos['fecha_fin'],
                "Fecha del Primer Devengado": datos['fecha_primer_devengado']
            }
            return
    
    # Si no existe, AGREGAR nuevo
    resultados.append({
        "CUI": cui,
        "Costo Inversión Total (a)": datos['costo_total'],
        "PIA 2026": datos['pia_2026'],
        "PIM 2026 (c)": datos['pim_2026'],
        "Devengado Acumulado al 2026 (b)": datos['devengado_acum'],
        "Devengado 2026 (d)": datos['devengado_2026'],
        "Fecha Inicio Ejecución": datos['fecha_inicio'],
        "Fecha Fin Ejecución": datos['fecha_fin'],
        "Fecha del Primer Devengado": datos['fecha_primer_devengado']
    })

def agregar_fallo(cui):
    """Agrega o mantiene un CUI como fallido"""
    for r in resultados:
        if str(r['CUI']) == str(cui):
            return  # Ya existe con NO DISPONIBLE, no hacer nada
    
    # Agregar nuevo con NO DISPONIBLE
    resultados.append({
        "CUI": cui,
        "Costo Inversión Total (a)": "NO DISPONIBLE",
        "PIA 2026": "NO DISPONIBLE",
        "PIM 2026 (c)": "NO DISPONIBLE",
        "Devengado Acumulado al 2026 (b)": "NO DISPONIBLE",
        "Devengado 2026 (d)": "NO DISPONIBLE",
        "Fecha Inicio Ejecución": "NO DISPONIBLE",
        "Fecha Fin Ejecución": "NO DISPONIBLE",
        "Fecha del Primer Devengado": "NO DISPONIBLE"
    })

# ================================================================
# LOOP PRINCIPAL
# ================================================================
tiempo_inicio = time.time()
exitosos = 0
fallos = 0

print(f"{'='*60}")
print(f"Inicio: {time.strftime('%H:%M:%S')}")
print(f"{'='*60}\n")

try:
    for idx, cui in enumerate(lista_cuis, 1):
        t_inicio = time.time()
        
        # Verificar si es reintento
        es_reintento = str(cui) in procesados_dict
        marca = "🔄" if es_reintento else "🆕"
        
        print(f"{marca} [{idx}/{len(lista_cuis)}] {cui}:", end=" ")
        
        try:
            datos = procesar_cui(cui)
            actualizar_resultado(cui, datos)
            exitosos += 1
            print(f"✅ ({time.time()-t_inicio:.1f}s)")
            
        except Exception as e:
            agregar_fallo(cui)
            fallos += 1
            print(f"❌ ({time.time()-t_inicio:.1f}s)")

        # GUARDAR CADA N CUIs
        if (idx % GUARDAR_CADA == 0) or (idx == len(lista_cuis)):
            guardar(resultados)
            if idx % GUARDAR_CADA == 0:
                print(f"   💾", end="")

        # Progreso cada 10 CUIs
        if idx % 10 == 0:
            t = time.time() - tiempo_inicio
            prom = t / idx
            eta = (len(lista_cuis) - idx) * prom
            print(f"\n   📊 {exitosos}✅ {fallos}❌ | {t:.0f}s | ETA:{eta:.0f}s\n")

except KeyboardInterrupt:
    print("\n\n⚠️ INTERRUMPIDO")
    guardar(resultados)
    driver.quit()
    print(f"💾 Progreso guardado. Ejecuta nuevamente para continuar.")
    exit()

# ================================================================
# FINALIZAR
# ================================================================
driver.quit()
guardar(resultados)

tiempo_total = time.time() - tiempo_inicio

print(f"\n{'='*60}")
print("🏁 COMPLETADO")
print(f"{'='*60}")
print(f"📊 Total procesados en sesión: {len(lista_cuis)}")
print(f"✅ Exitosos en sesión: {exitosos}")
print(f"❌ Fallidos en sesión: {fallos}")
print(f"⏱️ Tiempo: {tiempo_total:.1f}s ({tiempo_total/60:.1f} min)")
print(f"💾 Archivo: {RUTA_SALIDA}")

# Estadísticas finales del archivo completo
df_final = pd.DataFrame(resultados)
total_exitosos = len(df_final[df_final['Costo Inversión Total (a)'] != 'NO DISPONIBLE'])
total_fallidos = len(df_final[df_final['Costo Inversión Total (a)'] == 'NO DISPONIBLE'])

print(f"\n📊 ESTADÍSTICAS TOTALES DEL ARCHIVO:")
print(f"   Total CUIs: {len(df_final)}")
print(f"   ✅ Con datos: {total_exitosos} ({total_exitosos/len(df_final)*100:.1f}%)")
print(f"   ❌ Sin datos: {total_fallidos} ({total_fallidos/len(df_final)*100:.1f}%)")
print(f"{'='*60}")

📂 Cargando CUIs...
📋 Todos los CUIs pendientes: 937
⚡ Procesando 937 CUIs pendientes

Inicio: 23:45:49

🆕 [1/937] 2507146: ✅ (2.8s)
🆕 [2/937] 2654533: ✅ (2.9s)
🆕 [3/937] 2690987: ✅ (2.9s)
🆕 [4/937] 2638397: ✅ (3.0s)
🆕 [5/937] 2414624: ✅ (3.1s)
   💾🆕 [6/937] 2094808: ✅ (2.5s)
🆕 [7/937] 2573274: ✅ (3.0s)
🆕 [8/937] 2234314: ✅ (2.9s)
🆕 [9/937] 2697841: ✅ (2.9s)
🆕 [10/937] 2260211: ✅ (2.8s)
   💾
   📊 10✅ 0❌ | 29s | ETA:2676s

🆕 [11/937] 2539902: ✅ (2.8s)
🆕 [12/937] 2531607: ✅ (2.8s)
🆕 [13/937] 2192844: ✅ (2.9s)
🆕 [14/937] 2078503: ✅ (2.7s)
🆕 [15/937] 2525722: ✅ (2.9s)
   💾🆕 [16/937] 2354781: ✅ (3.0s)
🆕 [17/937] 2502896: ✅ (2.8s)
🆕 [18/937] 2271673: ✅ (2.8s)
🆕 [19/937] 2255793: ✅ (2.9s)
🆕 [20/937] 2427376: ✅ (3.0s)
   💾
   📊 20✅ 0❌ | 58s | ETA:2638s

🆕 [21/937] 2164051: ✅ (2.6s)
🆕 [22/937] 2502679: ✅ (2.9s)
🆕 [23/937] 2427400: ✅ (2.7s)
🆕 [24/937] 2183907: ✅ (2.7s)
🆕 [25/937] 2322531: ✅ (3.0s)
   💾🆕 [26/937] 2193990: ✅ (2.7s)
🆕 [27/937] 2654461: ✅ (2.7s)
🆕 [28/937] 2534855: ✅ (2.8s)
🆕 [29/937

## WEB SCRAPING PARA EXTRAER AVANCE FISICO %

In [3]:
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ================================================================
# CONFIGURACIÓN OPTIMIZADA
# ================================================================
RUTA_ENTRADA = r"C:\Users\user\Downloads\CUI_Todos.xlsx"
RUTA_SALIDA = r"C:\Users\user\Downloads\CUI_Todos2.xlsx"
MAX_REINTENTOS = 2
MODO_VISIBLE = True
GUARDAR_CADA = 5

# Timeouts optimizados
TIMEOUT_PAGINA = 20
TIMEOUT_ELEMENTO = 10

# ================================================================
# FUNCIONES DE CHECKPOINT
# ================================================================
def cargar_progreso():
    """Carga progreso previo si existe"""
    if os.path.exists(RUTA_SALIDA):
        try:
            df = pd.read_excel(RUTA_SALIDA)
            print(f"📥 Progreso encontrado: {len(df)} CUIs ya procesados")
            return df.to_dict('records')
        except:
            pass
    return []

def obtener_pendientes(completa, procesados):
    """Calcula CUIs pendientes"""
    if not procesados:
        return completa
    cuis_ok = {str(r['CUI']) for r in procesados}
    pendientes = [cui for cui in completa if str(cui) not in cuis_ok]
    if pendientes:
        print(f"⏳ Pendientes: {len(pendientes)}")
    return pendientes

def guardar(resultados):
    """Guarda progreso"""
    try:
        pd.DataFrame(resultados).to_excel(RUTA_SALIDA, index=False)
    except Exception as e:
        print(f"⚠️ Error guardando: {e}")

# ================================================================
# LEER CUIs
# ================================================================
print("📂 Cargando CUIs...")
df_cui = pd.read_excel(RUTA_ENTRADA)
lista_completa = df_cui['CUI'].astype(str).tolist()

resultados = cargar_progreso()
lista_cuis = obtener_pendientes(lista_completa, resultados)

if not lista_cuis:
    print("🎉 ¡Todos los CUIs ya fueron procesados!")
    exit()

# ================================================================
# CONFIGURAR NAVEGADOR OPTIMIZADO
# ================================================================
def crear_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()

    if not MODO_VISIBLE:
        options.add_argument("--headless=new")
    else:
        options.add_argument("--start-maximized")

    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-logging")
    options.add_argument("--log-level=3")
    options.add_argument("--silent")

    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.media_stream": 2,
        "profile.default_content_setting_values.geolocation": 2,
    }
    options.add_experimental_option("prefs", prefs)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.page_load_strategy = 'normal'

    driver = Chrome(service=service, options=options)
    driver.set_page_load_timeout(TIMEOUT_PAGINA)
    driver.set_script_timeout(15)
    return driver

driver = crear_driver()
print(f"⚡ Procesando {len(lista_cuis)} CUIs pendientes\n")

# ================================================================
# EXTRACCIÓN DE AVANCE FÍSICO
# ================================================================
def extraer_avance_fisico(driver):
    """Extrae el % Avance Físico de la página"""
    try:
        # Esperar a que cargue el elemento
        Wait(driver, TIMEOUT_ELEMENTO).until(
            EC.presence_of_element_located((By.ID, "por_afis"))
        )
        
        # Extraer el valor usando JavaScript
        avance = driver.execute_script("""
            var elem = document.getElementById('por_afis');
            if (elem) {
                var texto = elem.textContent || elem.innerText || '';
                return texto.trim();
            }
            return 'NO DISPONIBLE';
        """)
        
        # Validar que se obtuvo un valor
        if avance and avance != 'NO DISPONIBLE' and avance.strip():
            return avance
        else:
            return 'NO DISPONIBLE'
            
    except TimeoutException:
        print(f"⏱️ Timeout", end=" ")
        return 'NO DISPONIBLE'
    except Exception as e:
        print(f"⚠️ Error: {str(e)[:20]}", end=" ")
        return 'NO DISPONIBLE'

# ================================================================
# PROCESAR CUI
# ================================================================
def procesar_cui(cui, intento=1):
    try:
        # Construir URL directa con el CUI
        url = f"https://ofi5.mef.gob.pe/invierteWS/Repseguim/RepEstimac?codigo={cui}"
        driver.get(url)
        
        # Pequeña espera para que cargue la página
        time.sleep(0.5)
        
        # Extraer el avance físico
        avance = extraer_avance_fisico(driver)
        
        if avance == 'NO DISPONIBLE':
            raise Exception("No se pudo extraer el avance físico")
        
        return avance

    except Exception as e:
        if intento < MAX_REINTENTOS:
            print(f"🔄{intento+1}...", end=" ")
            time.sleep(1)
            return procesar_cui(cui, intento + 1)
        else:
            raise e

# ================================================================
# LOOP PRINCIPAL
# ================================================================
tiempo_inicio = time.time()
exitosos = 0
fallos = 0

print(f"{'='*60}")
print(f"Inicio: {time.strftime('%H:%M:%S')}")
print(f"{'='*60}\n")

try:
    for idx, cui in enumerate(lista_cuis, 1):
        t_inicio = time.time()
        print(f"[{idx}/{len(lista_cuis)}] {cui}:", end=" ")
        
        try:
            avance_fisico = procesar_cui(cui)
            
            resultados.append({
                "CUI": cui,
                "% Avance Físico": avance_fisico
            })
            exitosos += 1
            print(f"✅ {avance_fisico} ({time.time()-t_inicio:.1f}s)")
            
        except Exception as e:
            resultados.append({
                "CUI": cui,
                "% Avance Físico": "NO DISPONIBLE"
            })
            fallos += 1
            print(f"❌ ({time.time()-t_inicio:.1f}s)")

        # GUARDAR CADA N CUIs
        if (idx % GUARDAR_CADA == 0) or (idx == len(lista_cuis)):
            guardar(resultados)
            if idx % GUARDAR_CADA == 0:
                print(f"   💾", end="")

        # Progreso cada 10 CUIs
        if idx % 10 == 0:
            t = time.time() - tiempo_inicio
            prom = t / idx
            eta = (len(lista_cuis) - idx) * prom
            print(f"\n   📊 {exitosos}✅ {fallos}❌ | {t:.0f}s | ETA:{eta:.0f}s\n")

except KeyboardInterrupt:
    print("\n\n⚠️ INTERRUMPIDO")
    guardar(resultados)
    driver.quit()
    print(f"💾 Progreso guardado. Ejecuta nuevamente para continuar.")
    exit()

# ================================================================
# FINALIZAR
# ================================================================
driver.quit()
guardar(resultados)

tiempo_total = time.time() - tiempo_inicio

print(f"\n{'='*60}")
print("🏁 COMPLETADO")
print(f"{'='*60}")
print(f"📊 Total en sesión: {len(lista_cuis)}")
print(f"✅ Exitosos: {exitosos}")
print(f"❌ Fallidos: {fallos}")
print(f"⏱️ Tiempo: {tiempo_total:.1f}s")
print(f"💾 Archivo: {RUTA_SALIDA}")
print(f"{'='*60}")

📂 Cargando CUIs...
⚡ Procesando 592 CUIs pendientes

Inicio: 20:34:24

[1/592] 2507146: 🔄2... ❌ (2.2s)
[2/592] 2654533: 🔄2... ❌ (2.1s)
[3/592] 2690987: 🔄2... ❌ (2.1s)
[4/592] 2638397: 🔄2... ❌ (2.1s)
[5/592] 2414624: 🔄2... ❌ (2.1s)
   💾[6/592] 2094808: ✅ 86.28 % (0.6s)
[7/592] 2573274: 🔄2... ❌ (2.1s)
[8/592] 2234314: 🔄2... ❌ (2.1s)
[9/592] 2697841: 🔄2... ❌ (2.1s)
[10/592] 2260211: ✅ 70.97 % (0.6s)
   💾
   📊 2✅ 8❌ | 18s | ETA:1062s

[11/592] 2539902: 🔄2... ❌ (2.5s)
[12/592] 2531607: 🔄2... ❌ (2.1s)
[13/592] 2192844: ✅ 40.73 % (0.6s)
[14/592] 2078503: 🔄2... ❌ (2.1s)
[15/592] 2525722: ✅ 8.99 % (0.5s)
   💾[16/592] 2354781: ✅ 30.4 % (0.6s)
[17/592] 2502896: ✅ 6.31 % (0.6s)
[18/592] 2271673: ✅ 0.5 % (0.6s)
[19/592] 2255793: ✅ 88.75 % (0.6s)
[20/592] 2427376: ✅ 27.1 % (0.6s)
   💾
   📊 9✅ 11❌ | 29s | ETA:831s

[21/592] 2164051: ✅ 42 % (0.6s)
[22/592] 2502679: ✅ 12.8 % (0.6s)
[23/592] 2427400: ✅ 8.13 % (0.6s)
[24/592] 2183907: ✅ 75.52 % (0.6s)
[25/592] 2322531: ✅ 58.29 % (0.6s)
   💾[26/592] 21939